In [1]:
import geemap, ee

In [3]:
#Uncomment this when running for the first time and/or if the map doesnt render
# ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [4]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

This bit is needed if you're making changes to the qm lib file and want to reload it live

In [64]:
%reload_ext autoreload
import quality_mosaic as qm

# Choose date and location

## Define parameters

In [83]:
lat, lon = lat, lon = 33.869066751,-118.405718865
date = '2021-01-18'

## Define Hyperparameters

In [84]:
#define the total number of days before and after the date of interest you want to search``
max_search_window_months = 6
# define the number of days to median filtered S1 images
s1_median_days = 6

Convert to ee objects

In [99]:
point = ee.Geometry.Point([lon, lat])
sample = point.buffer(10).bounds()
roi = point.buffer(100).bounds()

In [75]:
visParamsRGB = {"min": 0, "max": 4000, "bands": ["B4", "B3", "B2"]}
visParamsVV  = {"min": -30, "max": 0, "bands": ["VV"]}

visParamsFScore  = {"min": -50, "max": 0, "bands": ["FScore"]}

green_blue_yellow_orange = ['#008080','#0039e6','#FFFF31','#f85d31']
visParamsMTGSI  = {'min':-0.5, 'max':0.25, 'palette':green_blue_yellow_orange, 'bands':['mTGSI']}

# Get mosaics for S1 and S2

In [79]:
s1_s2 = qm.get_s1_s2(roi=roi, date=date, max_search_window_months=6,s1_median_days=s1_median_days)

Search window from 2020-07-18 to 2021-07-18


In [80]:
s1_s2 = qm.addNDWI(qm.addBSI(qm.add_mTGSI(s1_s2)))

## Declare the bands you want to keep

In [97]:
keep_bands = ["B2","B3","B4","B8","B8A","B11","B12","VV","VH","mTGSI","BSI","NDWI"]
s1_s2 = s1_s2.select(keep_bands)
#print all the bands of s1_s2
print(s1_s2.bandNames().getInfo())

['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12', 'VV', 'VH', 'FScore', 'mTGSI', 'BSI', 'NDWI']


# Show interactive map

Add a marker manually if you want to change the location of the sample pointm

In [100]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(s1_s2, visParamsRGB, 'qm_s2')
Map.addLayer(s1_s2, visParamsVV, 's1')

Map.addLayer(s1_s2,visParamsMTGSI , 'mTGSI')

Map.addLayer(sample,
             {'color':'green', 'opacity':0.4},
             name='buffer')

Map.add_marker([lat, lon],tooltip='sample', name='sample', draggable=False)
Map.centerObject(sample, 20)

Map

Map(center=[33.869066767472795, -118.40571869467622], controls=(WidgetControl(options=['position', 'transparen…

### If you dropped a new pin, make sure to run this cell:

In [89]:
if (Map.draw_last_feature is not None):
    
    Map.draw_last_feature.geometry().getInfo()['coordinates']
    new_sample = Map.draw_last_feature.buffer(10).bounds()
    Map.addLayer(new_sample,
             {'color':'green', 'opacity':0.3},
             name='new_buffer')


### If you're happy with the location of the *most recent* pin, then run this cell:

In [101]:
#check if new_sample is defined
try:
    sample = new_sample
    #print the coordinates of the new pin
    print(Map.draw_last_feature.geometry().getInfo()['coordinates'])
except NameError:
    print ("You didnt drop a new marker")


AttributeError: 'NoneType' object has no attribute 'geometry'

## Extract all the band values of the area around the pin, aka sample

In [103]:
# Extract bands values
# Use reduceRegion to get a dictionary of means within sample
# Use getInfo() to get the dictionary from the server
sample = s1_s2.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': sample.geometry(),
    'scale': 10,
    'maxPixels': 1e9
}).getInfo()

sample


{'B11': 3779.028571428571,
 'B12': 3901.328979591836,
 'B2': 1973.8106122448976,
 'B3': 2525.9591836734694,
 'B4': 2944.963265306122,
 'B8': 3240.581224489795,
 'B8A': 3120.9485714285706,
 'BSI': 0.1273898981355593,
 'FScore': -3.7696249893078497,
 'NDWI': -0.12368247257228167,
 'VH': -27.987106069211258,
 'VV': -16.224042067078262,
 'mTGSI': 0.11267660659815573}